In [1]:
!pip install 'gluonts[torch]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.9 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.1/778.1 kB 20.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 2.2.2
    Uninstalling pytorch-lightning-2.2.2:
      Successfully uninstalled pytorch-lightning-2.2.2


In [2]:
!pip install optuna

In [3]:
import pandas as pd
import numpy as np
from gluonts.dataset.pandas import PandasDataset
from gluonts.torch.model.deepar import DeepAREstimator
#from gluonts.torch.distributions import NegativeBinomialOutput
#from gluonts.torch.distributions import PoissonOutput
from gluonts.torch.distributions import NormalOutput
from gluonts.evaluation import make_evaluation_predictions
import matplotlib.pyplot as plt
import json

# Split train and test
df = pd.read_csv('../input/wiki-data/Wiki_process_for_deepar.csv')
df.set_index('Date',inplace=True)
#df.index = pd.to_datetime(df.index).dt.strftime('%Y-%m-%d %H:%M:%S')
prediction_length = 15
freq = 'D'
split_date = pd.to_datetime('2016-12-17').strftime('%Y-%m-%d %H:%M:%S')
train = df[df.index < split_date]
test = df[df.index >= split_date]

In [6]:
train.shape,test.shape

((69849, 9), (2985, 9))

In [67]:
split_date1 = pd.to_datetime('2016-12-02').strftime('%Y-%m-%d %H:%M:%S')
train1 = df[df.index < split_date1]
val = train[train.index>=split_date1]

In [68]:
train1.shape,val.shape

((66864, 9), (2985, 9))

In [69]:
train1.head()

,Node,Value,Day_of_Week,Day_Of_Month,Month_Of_Year,Code,Agent,Access,State
Date,,,,,,,,,
2016-01-01 00:00:00,Total,156508,4,1,1,0,0,0,0
2016-01-01 00:00:00,de,15342,4,1,1,0,0,0,1
2016-01-01 00:00:00,en,63319,4,1,1,0,0,0,2
2016-01-01 00:00:00,fr,33489,4,1,1,0,0,0,3
2016-01-01 00:00:00,ja,17242,4,1,1,0,0,0,4


In [70]:
train1.reset_index(inplace=True)
val.reset_index(inplace=True)

train2 = train1.drop_duplicates(subset=['Node','Code','Agent','Access','State'])
train_static = pd.DataFrame({'State':train2['State'],
                             'Access':train2['Access'],
                             'Agent':train2['Agent'],
                             'Code':train2['Code'],
                             'Node':train2['Node']})
train_static.set_index('Node',inplace=True)

train_group = train1.groupby('Node')
standardized_params = {}
train_standard = train1.copy()



In [72]:
train_standard['Value'] = train_standard['Value'].astype(float)

In [73]:
train_standard

,Date,Node,Value,Day_of_Week,Day_Of_Month,Month_Of_Year,Code,Agent,Access,State
0,2016-01-01 00:00:00,Total,156508.0,4,1,1,0,0,0,0
1,2016-01-01 00:00:00,de,15342.0,4,1,1,0,0,0,1
2,2016-01-01 00:00:00,en,63319.0,4,1,1,0,0,0,2
3,2016-01-01 00:00:00,fr,33489.0,4,1,1,0,0,0,3
4,2016-01-01 00:00:00,ja,17242.0,4,1,1,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...
66859,2016-12-01 00:00:00,zh_MOB_AAG_005,15.0,3,1,12,146,24,18,6
66860,2016-12-01 00:00:00,zh_MOB_AAG_028,9.0,3,1,12,147,24,18,6
66861,2016-12-01 00:00:00,zh_MOB_AAG_031,151.0,3,1,12,148,24,18,6
66862,2016-12-01 00:00:00,zh_MOB_AAG_036,67.0,3,1,12,149,24,18,6


In [74]:
# 标准化
for cat,group in train_group:
    means = group['Value'].mean()
    stds = group['Value'].std()
    standardized_params[cat] = {'mean':means,'std':stds}
    train_standard.loc[group.index,'Value'] = (group['Value']-means)/stds

In [75]:
train_ds = PandasDataset.from_long_dataframe(train_standard.iloc[:,[0,1,2,3,4,5]],
                                             target="Value",
                                             timestamp='Date',
                                             freq='D',
                                             item_id="Node",
                                             feat_dynamic_real=['Day_of_Week','Day_Of_Month','Month_Of_Year'],
                                             static_features=train_static)

In [76]:
val_group = val.groupby('Node')
val_standard = val.copy()

val_standard['Value'] = val_standard['Value'].astype(float)
# 标准化
for cat,group in val_group:
    means = group['Value'].mean()
    stds = group['Value'].std()
    val_standard.loc[group.index,'Value'] = (group['Value']-means)/stds

In [77]:
val_standard

,Date,Node,Value,Day_of_Week,Day_Of_Month,Month_Of_Year,Code,Agent,Access,State
0,2016-12-02 00:00:00,Total,-0.569949,4,2,12,0,0,0,0
1,2016-12-02 00:00:00,de,1.006392,4,2,12,0,0,0,1
2,2016-12-02 00:00:00,en,-0.145353,4,2,12,0,0,0,2
3,2016-12-02 00:00:00,fr,0.219679,4,2,12,0,0,0,3
4,2016-12-02 00:00:00,ja,-0.589082,4,2,12,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...
2980,2016-12-16 00:00:00,zh_MOB_AAG_005,0.760371,4,16,12,146,24,18,6
2981,2016-12-16 00:00:00,zh_MOB_AAG_028,-0.286055,4,16,12,147,24,18,6
2982,2016-12-16 00:00:00,zh_MOB_AAG_031,3.590770,4,16,12,148,24,18,6
2983,2016-12-16 00:00:00,zh_MOB_AAG_036,-0.089982,4,16,12,149,24,18,6


In [78]:
val_ds = PandasDataset.from_long_dataframe(val_standard.iloc[:,[0,1,2,3,4,5]],
                                             target="Value",
                                             timestamp='Date',
                                             freq='D',
                                             item_id="Node",
                                             feat_dynamic_real=['Day_of_Week','Day_Of_Month','Month_Of_Year'],
                                             static_features=train_static)

In [83]:
from gluonts.torch.distributions import NormalOutput
estimator = DeepAREstimator(
            num_layers=3,
            hidden_size=40,
            lr=1e-3,
            prediction_length=prediction_length,
            context_length=10*prediction_length,
            patience = 10,
            freq='D',
            distr_output=NormalOutput(),
            scaling=False,
            num_parallel_samples=1000,
            batch_size=32,
            trainer_kwargs={
                "max_epochs": 1,
            },
        )

predictor = estimator.train(train_ds,num_worker = 4)
forecast_it = predictor.predict(val_ds)
# forecast_it, ts_it = make_evaluation_predictions(val_ds, predictor=predictor,num_samples=1000)
forecasts = list(forecast_it)

INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name  | Type        | Params | In sizes                                                         | Out sizes    
-------------------------------------------------------------------------------------------------------------------------
0 | model | DeepARModel | 39.0 K | [[1, 1], [1, 1], [1, 1242, 4], [1, 1242], [1, 1242], [1, 15, 4]] | [1, 1000, 15]
-------------------------------------------------------------------------------------------------------------------------
39.0 K    Trainable params
0         Non-trainable params
39.0 K    Total params
0.156     Total estimated model pa

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 50: 'train_loss' reached 1.08094 (best 1.08094), saving model to '/kaggle/working/lightning_logs/version_11/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO: `Trainer.fit` stopped: `max_epochs=1` reached.


In [85]:
node_nonsort = []
for i in range(len(forecasts)):
    node_nonsort.append(forecasts[i].item_id)
path = '../input/wiki-data/'
new_df = pd.read_csv(path+'Wiki_process.csv')
node_list = new_df.columns[1:].tolist()

In [86]:
index_list = [node_nonsort.index(i) for i in node_list]

In [46]:
val_standard.head()

,Date,Node,Value,Day_of_Week,Day_Of_Month,Month_Of_Year,Code,Agent,Access,State
0,2016-12-13 00:00:00,Total,0.006786,1,13,12,0,0,0,0
1,2016-12-13 00:00:00,de,0.177168,1,13,12,0,0,0,1
2,2016-12-13 00:00:00,en,-0.416581,1,13,12,0,0,0,2
3,2016-12-13 00:00:00,fr,1.093462,1,13,12,0,0,0,3
4,2016-12-13 00:00:00,ja,-0.045340,1,13,12,0,0,0,4


In [87]:
class DeepARTuningObjective:
    def __init__(
        self, dataset, prediction_length, freq, validation_label,val_ds,index_list
    ):
        self.dataset = dataset
        self.prediction_length = prediction_length
        self.freq = freq
        self.validation_label=validation_label
        self.val_ds = val_ds


    def get_params(self, trial) -> dict:
        return {
            "num_layers": trial.suggest_int("num_layers", 2, 5),
            "hidden_size": trial.suggest_int("hidden_size", 20, 50),
            "lr":trial.suggest_categorical('lr', choices=[1e-4,1e-3,1e-2]),
            "batch_size": trial.suggest_int("batch_size", low= 16, high =64, step =16)
        }

    def __call__(self, trial):
        params = self.get_params(trial)
        estimator = DeepAREstimator(
            num_layers=params["num_layers"],
            hidden_size=params["hidden_size"],
            lr=params["lr"],
            prediction_length=self.prediction_length,
            context_length=10*prediction_length,
            patience = 10,
            freq=self.freq,
            distr_output=NormalOutput(),
            scaling=False,
            num_parallel_samples=1000,
            batch_size=params["batch_size"],
            trainer_kwargs={
                "max_epochs": 20,
            },
        )

        predictor = estimator.train(self.dataset, cache_data=True)
        forecast_it, ts_it = make_evaluation_predictions(dataset=self.val_ds, predictor=predictor,num_samples=1000)

        forecasts = list(forecast_it)
        
        samples_total = []
        for j in range(15):
            samples_step = []
            for i in range(len(forecasts)):
                samples_step.append(forecasts[i].samples[:,j].reshape((1,1000)))
            samples_1 = []
            for k in index_list:
                samples_1.append(samples_step[k])
            samples_total.append(np.concatenate(samples_1,axis=0))
        samples_new = np.concatenate(samples_total,axis=1)
        
        real = []
        for i in range(15):
            real_step = np.array(val_standard.iloc[i*199:(i+1)*199,2].tolist())
            repeated_arr = np.repeat(real_step, 1000, axis=0)
            two_dim_array = repeated_arr.reshape(199, 1000).transpose()
            real.append(two_dim_array)
        real_2 = np.concatenate(real,axis=0).transpose()
        
        samples_new2 = samples_new.copy()
        samples_new2 = samples_new2.transpose()
        np.random.shuffle(samples_new2)
        samples_new2 = samples_new2.transpose()
        
        dif1 = samples_new - samples_new2
        dif2 = real_2 - samples_new
        term1 = np.sum(np.square(dif1))
        term2 = np.sum(np.square(dif2))
        res1 = ((-0.5*term1)+term2)/(real_2.shape[1])
        
        return res1

In [88]:
import time
import optuna

start_time = time.time()
study = optuna.create_study(direction="minimize")
study.optimize(
    DeepARTuningObjective(
        train_ds, prediction_length, freq,val_standard, val_ds, index_list
    ),
    n_trials=10,
)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
print(time.time() - start_time)

[I 2024-05-06 14:26:17,813] A new study created in memory with name: no-name-b79122ef-accb-4467-8b34-8ce3cfbe37d4
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name  | Type        | Params | In sizes                                                         | Out sizes    
-------------------------------------------------------------------------------------------------------------------------
0 | model | DeepARModel | 35.5 K | [[1, 1], [1, 1], [1, 1242, 4], [1, 1242], [1, 1242], [1, 15, 4]] | [1, 1000, 15]
-------------------------------------------------------------------------------------------------------------------------
3

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 50: 'train_loss' reached 1.03214 (best 1.03214), saving model to '/kaggle/working/lightning_logs/version_12/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO: Epoch 1, global step 100: 'train_loss' reached 0.91387 (best 0.91387), saving model to '/kaggle/working/lightning_logs/version_12/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO: Epoch 2, global step 150: 'train_loss' reached 0.85859 (best 0.85859), saving model to '/kaggle/working/lightning_logs/version_12/checkpoints/epoch=2-step=150.ckpt' as top 1
INFO: Epoch 3, global step 200: 'train_loss' reached 0.81463 (best 0.81463), saving model to '/kaggle/working/lightning_logs/version_12/checkpoints/epoch=3-step=200.ckpt' as top 1
INFO: Epoch 4, global step 250: 'train_loss' reached 0.77626 (best 0.77626), saving model to '/kaggle/working/lightning_logs/version_12/checkpoints/epoch=4-step=250.ckpt' as top 1
INFO: Epoch 5, global step 300: 'train_loss' reached 0.73552 (best 0.73552), saving model to '/kaggle/

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 50: 'train_loss' reached 1.23902 (best 1.23902), saving model to '/kaggle/working/lightning_logs/version_13/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO: Epoch 1, global step 100: 'train_loss' reached 1.17436 (best 1.17436), saving model to '/kaggle/working/lightning_logs/version_13/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO: Epoch 2, global step 150: 'train_loss' reached 1.16375 (best 1.16375), saving model to '/kaggle/working/lightning_logs/version_13/checkpoints/epoch=2-step=150.ckpt' as top 1
INFO: Epoch 3, global step 200: 'train_loss' reached 1.08552 (best 1.08552), saving model to '/kaggle/working/lightning_logs/version_13/checkpoints/epoch=3-step=200.ckpt' as top 1
INFO: Epoch 4, global step 250: 'train_loss' reached 1.04809 (best 1.04809), saving model to '/kaggle/working/lightning_logs/version_13/checkpoints/epoch=4-step=250.ckpt' as top 1
INFO: Epoch 5, global step 300: 'train_loss' reached 1.04557 (best 1.04557), saving model to '/kaggle/

...

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 50: 'train_loss' reached 1.05573 (best 1.05573), saving model to '/kaggle/working/lightning_logs/version_20/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO: Epoch 1, global step 100: 'train_loss' reached 0.89814 (best 0.89814), saving model to '/kaggle/working/lightning_logs/version_20/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO: Epoch 2, global step 150: 'train_loss' reached 0.84869 (best 0.84869), saving model to '/kaggle/working/lightning_logs/version_20/checkpoints/epoch=2-step=150.ckpt' as top 1
INFO: Epoch 3, global step 200: 'train_loss' reached 0.82166 (best 0.82166), saving model to '/kaggle/working/lightning_logs/version_20/checkpoints/epoch=3-step=200.ckpt' as top 1
INFO: Epoch 4, global step 250: 'train_loss' reached 0.78345 (best 0.78345), saving model to '/kaggle/working/lightning_logs/version_20/checkpoints/epoch=4-step=250.ckpt' as top 1
INFO: Epoch 5, global step 300: 'train_loss' reached 0.73417 (best 0.73417), saving model to '/kaggle/

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 50: 'train_loss' reached 1.21555 (best 1.21555), saving model to '/kaggle/working/lightning_logs/version_21/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO: Epoch 1, global step 100: 'train_loss' reached 1.13782 (best 1.13782), saving model to '/kaggle/working/lightning_logs/version_21/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO: Epoch 2, global step 150: 'train_loss' reached 1.07067 (best 1.07067), saving model to '/kaggle/working/lightning_logs/version_21/checkpoints/epoch=2-step=150.ckpt' as top 1
INFO: Epoch 3, global step 200: 'train_loss' reached 1.04431 (best 1.04431), saving model to '/kaggle/working/lightning_logs/version_21/checkpoints/epoch=3-step=200.ckpt' as top 1
INFO: Epoch 4, global step 250: 'train_loss' reached 0.98134 (best 0.98134), saving model to '/kaggle/working/lightning_logs/version_21/checkpoints/epoch=4-step=250.ckpt' as top 1
INFO: Epoch 5, global step 300: 'train_loss' was not in top 1
INFO: Epoch 6, global step 350: 'train_l

Number of finished trials: 10
Best trial:
  Value: 180.60188403658893
  Params: 
    num_layers: 2
    hidden_size: 35
    lr: 0.001
    batch_size: 64
240.51415395736694
